In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-3/train.csv')
df_test=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-3/test.csv')

In [ ]:
df_train.count()

In [ ]:
df=pd.concat([df_train,df_test])

In [ ]:
df.iloc[22944]

In [ ]:
df.iloc[22950]

In [ ]:
df_test.iloc[0]

In [ ]:
df.Province_State.fillna('NaN',inplace=True)

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
enc=OneHotEncoder()
oe=OrdinalEncoder()

In [ ]:
 df[['Province_State','Country_Region']] = oe.fit_transform(df.loc[:,['Province_State','Country_Region']])

In [ ]:
df_ttrain=df.iloc[0:22950,:]

In [ ]:
df_ttrain.shape

In [ ]:
df_ttest=df.iloc[22950:,:]

In [ ]:
df_ttest.shape

In [ ]:
df_ttrain['Date']=pd.to_datetime(df_ttrain['Date'])

In [ ]:
df_ttest['Date']=pd.to_datetime(df_ttest['Date'])

In [ ]:
import datetime as dt
def create_features(df):
    df['day'] = df['Date'].dt.day
    df['month'] = df['Date'].dt.month
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['quarter'] = df['Date'].dt.quarter
    df['weekofyear'] = df['Date'].dt.weekofyear
    return df

In [ ]:
df_ttrain.head()

In [ ]:
df_ttrain=df_ttrain.drop(['ForecastId'],axis=1)

In [ ]:
df_ttest=df_ttest.drop(['Id','Fatalities','ConfirmedCases'],axis=1)

In [ ]:
df_ttrain.shape

In [ ]:
df_ttrain = create_features(df_ttrain)

In [ ]:
df_ttrain.head()

In [ ]:
df_ttest = create_features(df_ttest)

In [ ]:
df_ttest.head()

In [ ]:
from xgboost.sklearn import XGBRegressor

In [ ]:
  model1 = XGBRegressor(n_estimators=3000)

In [ ]:
X_train=df_ttrain.loc[:,['Country_Region','Province_State','day','month','dayofweek','dayofyear','quarter','weekofyear']].values

In [ ]:
y_train1=df_ttrain.loc[:,['ConfirmedCases']].values

In [ ]:
y_train2=df_ttrain.loc[:,['Fatalities']].values

In [ ]:
model1.fit(X_train,y_train1)

In [ ]:
  model2 = XGBRegressor(n_estimators=3000)

In [ ]:
model2.fit(X_train,y_train2)

In [ ]:
X_test=df_ttest.loc[:,['Country_Region','Province_State','day','month','dayofweek','dayofyear','quarter','weekofyear']].values

In [ ]:
y_pred1=model1.predict(X_test)

In [ ]:
y_pred2=model2.predict(X_test)

In [ ]:
submission1=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-3/submission.csv')

In [ ]:
submission1['ConfirmedCases']=y_pred1
submission1['Fatalities']=y_pred2

In [ ]:
submission1.head()

In [ ]:
submission1.to_csv('submission.csv',index=False)